                                 TIDY DATA EXERCISES
Do your work for this exercise in a jupyter notebook or python script named tidy_data.
Save this work in your classification-exercises repo. Add, commit, and push your changes.

1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student.
One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

You should end up with something like this:


In [1]:
#name
#Billy    0.5250
#Jane     0.6875
#John     0.9125
#Sally    0.7625
#Name: grade, dtype: float64

In [2]:
import pandas as pd
import numpy as np
import pydataset as data
import env
import os

database_url_base = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/tidy_data'

In [3]:
df = pd.read_sql('SELECT * FROM attendance', database_url_base)
df


,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [4]:
# Half day - H to = .5 (50%),
# Absent - A to = 0,
# Tardy - T>= 10 = A 
# and Present - P to = 1
# then take the average.

In [5]:
df = df.melt(id_vars = 'Unnamed: 0')
df.head(10)

,Unnamed: 0,variable,value
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


In [6]:
df = df.rename(columns = {'Unnamed: 0' : 'student', 'variable': 'date', 'value' : 'grade'})
df.head()

,student,date,grade
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [7]:
df['grade'] = df.grade.map({'A': 0.0, 'H': 0.5, 'T': 0.9, 'P':1.0})

In [82]:
df.head()

,student,date,grade
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9


In [83]:
df.groupby(by = 'student').grade.mean()
#name
#Billy    0.5250
#Jane     0.6875
#John     0.9125
#Sally    0.7625
#Name: grade, dtype: float64

student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64

In [84]:
# pivot table

df.pivot_table(index = ['student'], values = ['grade'])

,grade
student,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


2. Coffee Levels

a. Read the coffee_levels table.

In [72]:
df = pd.read_sql('SELECT * FROM coffee_levels', database_url_base)
df.head(10)

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


b. Transform the data so that each carafe is in it's own column.


In [73]:
df.pivot_table(index = 'hour', columns = 'coffee_carafe' ,values = 'coffee_amount').reset_index()


coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


3. Cake Recipes

a. Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes,
 oven rack positions, and oven temperatures.


In [90]:
df = pd.read_sql('SELECT * FROM cake_recipes', database_url_base)

In [91]:
df.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


b. Tidy the data as necessary.

In [93]:
df[['recipe', 'position']] = df['recipe:position'].str.split(':' , expand = True)
df.head()

,recipe:position,225,250,275,300,recipe,position
0,a:bottom,61.738655,53.912627,74.414730,98.786784,a,bottom
1,a:top,51.709751,52.009735,68.576858,50.228470,a,top
2,b:bottom,57.095320,61.904369,61.196980,99.248541,b,bottom
3,b:top,82.455004,95.224151,98.594881,58.169349,b,top
4,c:bottom,96.470207,52.001358,92.893227,65.473084,c,bottom


In [94]:
df = df.drop(columns = 'recipe:position')

In [95]:
df_melt = df.melt(id_vars = ['recipe', 'position'], var_name = 'temp', value_name = 'score')

In [96]:
df_melt.head()

,recipe,position,temp,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


c. Which recipe, on average, is the best? recipe b.

In [ ]:
# recipe b is the best on average

In [97]:
df_melt.groupby(by = 'recipe').mean()

,score
recipe,
a,63.922201
b,76.736074
c,75.874748
d,62.864844


d. Which oven temperature, on average, produces the best results? 275


In [ ]:
# oven temp 275 produces the best results on average

In [98]:
df_melt.groupby(by = 'temp').mean()

,score
temp,
225,71.306022
250,66.577437
275,74.886754
300,66.627655


e. Which combination of recipe, rack position, and temperature gives the best result?
recipe b, bottom rack, 300 degrees

In [ ]:
# combination of recipe, rack position & temp 300 degrees gives the best results.

In [100]:
df_melt.sort_values('score', ascending = False).head(5)

,recipe,position,temp,score
26,b,bottom,300,99.248541
24,a,bottom,300,98.786784
19,b,top,275,98.594881
7,d,top,225,96.873178
4,c,bottom,225,96.470207
